In [91]:
import torch

In [92]:
print(torch.__version__)

2.3.0+cu121


In [93]:
print(torch.cuda.is_available())

False


# **Scalers, vectors, matrices and tensors**

In [94]:
import numpy as np

tensor0d = torch.tensor(2)

tensor1d = torch.tensor([2,3])

#2d tensor from nested python list
tensor2d = torch.tensor([[1,1],[2,2]])

#3d tensor from nested python list
tensor3d = torch.tensor([[[1,1],[2,2]],[[3,3],[1,1]]])

#create 3d tensor from numpy

array3 = np.array([[[1,1],[2,2]],
                   [[3,3],[1,1]]])
tensor3d_2 = torch.tensor(array3) # copies numpy array
tensor3d_3 = torch.from_numpy(array3) #shares memory with numpy array



In [95]:
array3[0,0,0] = 999
print(tensor3d_2) #remains unchanged

tensor([[[1, 1],
         [2, 2]],

        [[3, 3],
         [1, 1]]])


In [96]:
print(tensor3d_3) #changes because of memory sharing

tensor([[[999,   1],
         [  2,   2]],

        [[  3,   3],
         [  1,   1]]])


**Tensor data types**

In [97]:
tensor1d = torch.tensor([1,2,2])
print(tensor1d.dtype)

torch.int64


In [98]:
floatvec = torch.tensor([1.0,2.0,3.0])
print(floatvec.dtype)

torch.float32


In [99]:
floatvec = tensor1d.to(torch.float32)
floatvec.dtype

torch.float32

**Common pytorch tensor operations**

In [100]:
tensor2d = torch.tensor([[1,2,3],[4,5,6]])
tensor2d

tensor([[1, 2, 3],
        [4, 5, 6]])

In [101]:
tensor2d.shape

torch.Size([2, 3])

In [102]:
tensor2d.reshape(3,2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [103]:
tensor2d.view(3,2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [104]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [105]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 32],
        [32, 77]])

In [106]:
tensor2d @ tensor2d.T

tensor([[14, 32],
        [32, 77]])

In [107]:
import torch.nn.functional as F

y= torch.tensor([1.0]) #true label
x1 = torch.tensor([1.1]) #input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0]) #bias unit

z = x1*w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a,y)
print(loss)

tensor(0.0852)


**Automatic Differentiation**

In [108]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0]) #true label
x1 = torch.tensor([2.0]) #input feature
w1 = torch.tensor([1.2],requires_grad = True) #weight bias
b = torch.tensor([0.0],requires_grad = True)
z = x1*w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a,y)
grad_L_w1 = grad(loss, w1,retain_graph=True)
grad_L_b = grad(loss,b,retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.1663]),)
(tensor([-0.0832]),)


In [109]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.1663])
tensor([-0.0832])


**Implementing multilayer neural network**

In [110]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self,num_inputs,num_outputs):
    super().__init__()

    self.layers = torch.nn.Sequential(
        #1st hidden layer
        torch.nn.Linear(num_inputs,30),
        torch.nn.ReLU(),

        #2nd hidden layer
        torch.nn.Linear(30,20),
        torch.nn.ReLU(),

        #output layer
        torch.nn.Linear(20,num_outputs),
    )

  def forward(self,x):
      logits = self.layers(x)
      return logits

In [111]:
model = NeuralNetwork(50,3)

In [112]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [113]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters", num_params)

Total number of trainable model parameters 2213


In [114]:
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        [-0.0920, -0.0480,  0.0105,  ..., -0.0923,  0.1201,  0.0330],
        ...,
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509],
        [-0.1250,  0.0513,  0.0366,  ..., -0.1370,  0.1074, -0.0704]],
       requires_grad=True)


In [115]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [116]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [117]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [118]:
with torch.no_grad():
  out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [119]:
with torch.no_grad():
  out = torch.softmax(model(X),dim=1)

print(out)

tensor([[0.3113, 0.3934, 0.2952]])


**setting up efficient data loaders**

In [120]:
X_train = torch.tensor([
    [-1.2, 3.0],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]

])
y_train = torch.tensor([0, 0, 0, 1, 1])

In [121]:
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [122]:
from torch.utils.data import Dataset
class ToyDataset(Dataset):
  def __init__(self,X,y):
    self.features = X
    self.labels = y
  def __getitem__(self,index):
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y
  def __len__(self):
    return self.labels.shape[0]

train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test,y_test)

In [123]:
len(train_ds)

5

In [130]:
from torch.utils.data import DataLoader

torch.manual_seed(123)
train_loader = DataLoader(
    dataset = train_ds,
    batch_size =2,
    shuffle =True,
    num_workers=0
)

In [131]:
test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers =0
)

In [132]:
for idx, (x,y) in enumerate(train_loader):
  print(f"Batch {idx+1} : ", x,y)


Batch 1 :  tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2 :  tensor([[-1.2000,  3.0000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3 :  tensor([[ 2.7000, -1.5000]]) tensor([1])


In [133]:
train_loader = DataLoader(
    dataset = train_ds,
    batch_size =2,
    shuffle = True,
    num_workers= 0,
    drop_last = True
)

In [134]:
for idx,(x,y) in enumerate(train_loader):
  print(f"Batch {idx+1}: ", x,y)

Batch 1:  tensor([[-1.2000,  3.0000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2:  tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


**A typical training loop**

In [136]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2,num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(),lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features,labels) in enumerate(train_loader):
    logits = model(features)
    loss = F.cross_entropy(logits,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #logging
    print(f"Epoch: {epoch+1:03d}/num_epochs:03d"
    f"| Batch: {batch_idx:03d}/{len(train_loader):03d}"
    f"| Train/Val Loss: {loss:.2f}")
  model.eval()

Epoch: 001/num_epochs:03d| Batch: 000/002| Train/Val Loss: 0.75
Epoch: 001/num_epochs:03d| Batch: 001/002| Train/Val Loss: 0.65
Epoch: 002/num_epochs:03d| Batch: 000/002| Train/Val Loss: 0.44
Epoch: 002/num_epochs:03d| Batch: 001/002| Train/Val Loss: 0.12
Epoch: 003/num_epochs:03d| Batch: 000/002| Train/Val Loss: 0.03
Epoch: 003/num_epochs:03d| Batch: 001/002| Train/Val Loss: 0.00


In [137]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)

print(outputs)

tensor([[ 2.7776, -4.0477],
        [ 2.5311, -3.7465],
        [ 2.0880, -3.1744],
        [-1.4819,  1.4822],
        [-1.7182,  1.7348]])


In [138]:
torch.set_printoptions(sci_mode =False)
probas = torch.softmax(outputs,dim=1)
print(probas)


predictions = torch.argmax(outputs,dim=1)
print(predictions)

tensor([[0.9989, 0.0011],
        [0.9981, 0.0019],
        [0.9948, 0.0052],
        [0.0491, 0.9509],
        [0.0307, 0.9693]])
tensor([0, 0, 0, 1, 1])


In [139]:
predictions == y_train

tensor([True, True, True, True, True])

In [140]:
torch.sum(predictions==y_train)

tensor(5)

In [141]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [142]:
compute_accuracy(model, train_loader)

1.0